# QA Rewriting with GPT-3.5 Turbo

In [1]:
import time
import json
from tqdm import tqdm

import pandas as pd
import numpy as np
from typing import Union

import openai
from openai import AsyncOpenAI
from openai import OpenAI

import asyncio
import nest_asyncio

## Import Data

In [2]:
with open('../../data/aha_qa/a_ha_original.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data).transpose()
df.head()

,urls,title,question,answer
0,https://www.a-ha.io/questions/4cb726a43f6fea7f...,상법에서 정한 기한을 넘어선 주주총회의사록 공증 진행시 효력이 있는지요?,"안녕하세요,주주총회를 진행한 이후 일정 기간안에 법무사를 통한 공증을 진행해야 하는...",[안녕하세요. 이성재 변호사입니다.주주총회 결의사항 중 등기사항이 없다면 공증을 받...
1,https://www.a-ha.io/questions/4a8035d424b93c9e...,형사사법포털에는 기소유예가 떴는데 아직 어떠한 연락도 안와서요 혹시 왜그런건가요?,점유이탈횡령죄로 조사받고 얼마전 2일 기소유예 처분을 받았습니다. 근데 연락 받아서...,[안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내...
2,https://www.a-ha.io/questions/496d83cb9dfc2408...,자본금 10억 미만의 법인회사에서 이사가 1명일 때 발생하는 문제점,"안녕하세요,자본금 10억 미만의 회사지만 외부에서 투자를 많이 받고 있습니다.자본금...","[10억 미만의 회사이고 이사가 과반주주권자라면,주주총회 등을 적법하게 통지하고 개..."
3,https://www.a-ha.io/questions/4860dd4377505d30...,모든 계약은 법적으로 15일이내 해지가 가능한지요~?,한 회사(판매 사업)에 지사 계약을 일주일전에 했는데 할수없는상황이 되서 해지를 하...,[모든 계약에 대해서는 15일 만에 해지가 가능하다고 하는 규정은 따로 없습니다해당...
4,https://www.a-ha.io/questions/4afe128df4bd0726...,항소심 재판은 어떻게 진행되는건가요?,아는 지인이 항소후에 항소심 재판이 열리는데 어떻게 진행되는지 궁금합니다. 혹시 변...,[안녕하세요? 아하(Aha) 법률 상담 지식답변자 김성훈 변호사입니다. 질문하신 내...


In [3]:
with open('../prompt/AHA_Prompt.txt', 'r') as f:
    prompt_file = f.read()

prompt_file

'### instruction ###\n깔끔하고 핵심을 잘 알 수 있는 질문과 답변 쌍을 만들고자 한다.\n\n주어진 제목과 본문을 보고, 이를 하나의 질문 형태로 만들어라.\n생성된 질문에는 제목과 본문에 대한 내용이 모두 포함되어 있어야 한다.\n\n그리고 기존의 답변을 참고해서 새롭게 만든 질문에 대한 새로운 답변을 만들어라.\n\n이때 생성 형태는 다음과 같아야 한다.\n\n새로운 질문 및 답변:\n  질문:\n  답변:\n\n### input ###\n제목: {}\n본문: {}\n대답: {}\n\n==========\n새로운 질문 및 답변:'

## OpenAI API

In [4]:
nest_asyncio.apply()

client = AsyncOpenAI(
    api_key = '',  # Text_Analytics
    organization=''  # DSBA
)

In [5]:
# gpt-3.5-turbo 모델로 답변 생성

async def gpt_generate(title, body, original_answer):
    prompt = prompt_file.format(title, body, original_answer)
    
    completion = await client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"{prompt}"}
    ]
    )

    return completion.choices[0].message.content

In [6]:
# 생성한 답변을 기반으로 Cleaned Q&A 프롬프트 생성

def make_dict(response, url):
    split_text = response.split("\n")
    question_text = split_text[0].split("질문: ")[1] if len(split_text[0].split("질문: ")) > 1 else ""
    answer_text = split_text[1].split("답변: ")[1] if len(split_text[1].split("답변: ")) > 1 else ""

    qa_dict = {
        "instruction": question_text,
        "output": answer_text,
        "url": url
    }

    return qa_dict

In [12]:
# 모든 데이터에 대해서 반복

def make_qa_pair(df):
    qa_pair = {}

    for i in tqdm(df.tail().iterrows()):
        loop = asyncio.get_event_loop()
        result = loop.run_until_complete(gpt_generate(i[1]['title'], i[1]['question'], i[1]['answer']))

        # qa_dict = make_dict(result, i[1]['urls'])
        qa_pair[i[0]] = result
        print(' ', i[0], 'finished!')
        print('---------------------------------')
    
    return qa_pair

In [13]:
qa_pair = make_qa_pair(df)

In [14]:
qa_pair

{'111330': {'instruction': '전세금을 돌려받지 못할 경우 법률적으로 1순위가 되는 것인가요? 그리고 법적으로 전세금을 되찾기 위한 절차는 무엇인가요?',
  'output': '안녕하십니까. 전입신고와 확정일자를 받았고 물권(저당권 등이 없다면 1순위가 됩니다. 따라서 보증금 미반환시 법원에 보증금 반환 소송을 진행하여 확정판결을 받은 후 부동산에 대한 강제경매를 진행할 수 있습니다. 그리고 돌려받지 못할 경우 임대인의 다른 부동산, 동산 등에 대해서도 강제집행을 할 수 있습니다(다만 미리 가압류, 가처분 등이 필요할 수 있습니다). 감사합니다.',
  'url': 'https://www.a-ha.io/questions/4a32d8fe397217f5bb13e46279e85597'},
 '111331': {'instruction': '공증을 받은 사서증서에 강제집행 인낙의 표시가 있는 경우, 어떻게 법적으로 효력이 발생하나요?',
  'output': '당사자가 금전소비대차를 함에 있어 강제집행 인낙의 표시가 있는 공정증서를 작성한 후 공증을 받으면, 법원의 확정판결 없이도 바로 강제집행을 할 수 있습니다. 이는 공증인이 진정한 작성명의인의 의사에 따라 작성된 것임을 확인하는 과정으로, 법적 효력이 발생합니다.',
  'url': 'https://www.a-ha.io/questions/4e414d01a0773f29b7773ab548a986bb'},
 '111332': {'instruction': '전세로 살던 집이 경매로 매각되어 임차인이 일부만 보증금을 돌려받은 경우, 임차인은 어떻게 남은 전세보증금을 회수할 수 있을까요? 그리고 회수를 위한 절차는 무엇인가요?',
  'output': '다음 답변을 참고하여, 임차인은 남은 전세보증금을 회수하기 위해 소송을 통해 채권을 확보할 수 있습니다. 이를 위해 임차인은 다른 부동산이나 동산에 대한 보전처분(가압류, 가처분)을 신청한 후, 본안 소송을 제기하여 확정이 된다면 집행 절차에 착수하면 됩

In [15]:
with open('../../data/aha_qa/qa_pairs.json', 'w') as f:
    json.dump(qa_pair, f, ensure_ascii=False, indent=4)

# Merge JSON files

In [2]:
# json 파일 하나로 합치기

directory = '../../data/aha_qa'

qa_pair_files = [file for file in os.listdir(directory) if file.startswith('qa_pair') and file.endswith('.json')]

combined_data = {}

for file in qa_pair_files:
    file_path = os.path.join(directory, file)
    with open(file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
        combined_data.update(data)

combined_output_file = os.path.join(directory, 'combined_qa_pairs.json')

with open(combined_output_file, 'w', encoding='utf-8') as output_json_file:
    json.dump(combined_data, output_json_file, indent=4, ensure_ascii=False)

In [3]:
with open('../../data/aha_qa/combined_qa_pairs.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [11]:
import re

# 데이터 후처리
def process_data(data):
    sorted_data = dict(sorted(data.items(), key=lambda item: int(item[0])))
    
    processed_data = {}
    
    err_cnt = 0
    for key, value in sorted_data.items():
        try:
            parts = re.split(r'질문\s*:', value['answer'])
            
            try:
                question_part = re.split(r'답변\s*:', parts[1])
                question = question_part[0].strip()
                answer = question_part[1].strip()
            except IndexError:
                question_part = re.split(r'대답\s*:', parts[1])
                question = question_part[0].strip()
                answer = question_part[1].strip()

            question = question.replace('\n', '')
            answer = answer.replace('\n', '')

            processed_data[key] = {
                'question': question,
                'answer': answer,
                'url': value['url']
            }
        except:
            err_cnt += 1
    
    print(f'Error total count: {err_cnt}')

    return processed_data

In [12]:
qa_pairs = process_data(data)

Error total count: 111334


In [5]:
# 답변 데이터 후처리

def remove_text(json_obj):
    pattern = r"\s*안녕하세요.*?변호사입니다\.\s*"
    if isinstance(json_obj, dict):
        for key in json_obj:
            json_obj[key] = remove_text(json_obj[key])
    elif isinstance(json_obj, list):
        json_obj = [remove_text(item) for item in json_obj]
    elif isinstance(json_obj, str):
        json_obj = re.sub(pattern, '', json_obj)
    return json_obj

In [8]:
cleaned_data = remove_text(data)
cleaned_data

{'75000': {'question': '이혼 후에 단독으로 아파트 공동명의를 변경하는 것이 더 나은가요? 이 경우에는 어떤 세금이 발생하나요?',
  'answer': '이혼으로 재산분할을 하면서 명의를 변경하시는 것이 보다 타당한 수순으로 보입니다. 구체적인 세금의 액수는 답변이 어렵습니다.',
  'url': 'https://www.a-ha.io/questions/454862b7b16d7456abfa757a500b01e4'},
 '75001': {'question': '신차나 중고차를 전액할부로 구입할 때 초기 비용을 내야할까요?',
  'answer': '안녕하세요. 초기 비용은 차량을 전액할부로 구입할 때 자동차 등록세, 취등록비, 보험료 등의 초기 비용이 발생하게 됩니다. 하지만 전액할부의 경우 선수금을 내지 않고 할부로만 차량 가격을 지불하게 됩니다. 또한, 이 부분은 신용 등급에 따라 상당한 이자 비용이 발생할 수 있으니 주의하셔야 합니다.',
  'url': 'https://www.a-ha.io/questions/476ef9b931755b75a9df98775676f3d3'},
 '75002': {'question': '법적으로 가족관계를 끊는 방법이 존재할까요?',
  'answer': '혈연으로 이어진 관계에 대하여 오기 등이 아닌 절연했다는 사유만으로 가족관계를 단절시키는 절차는 현행법상 마련되어 있지 않습니다.',
  'url': 'https://www.a-ha.io/questions/43243efefc19b23ea36deafc81baebdf'},
 '75003': {'question': 'Sns에서 돈을 불려주는 사기 피해를 입었을 경우, 피해를 본 사람이 처벌을 받을 가능성은 있을까요?',
  'answer': '안녕하세요. 황성필 변호사/경제·금융전문가입니다. 구체적인 사실관계에 따라 달라질 수 있지만, 돈과 계좌를 함께 보냈다면 문제될 소지가 있을 수 있습니다. 이에 대한 상세한 상황을 파악하여야 합니다.',
  'url': 'h

# Train-Test Split

In [1]:
import json
import random

from sklearn.model_selection import train_test_split

In [3]:
data_list = list(cleaned_data.items())

train_data, test_data = train_test_split(data_list, test_size=10000, random_state=42)

In [11]:
with open('../../data/train_data.json', 'w') as f:
    json.dump(dict(train_data), f, ensure_ascii=False, indent=4)

In [12]:
with open('../../data/test_data.json', 'w') as f:
    json.dump(dict(test_data), f, ensure_ascii=False, indent=4)

----